# How can we improve this?

Let's take a look at the code we've got so far.

The first call we do, for the healthcheck, is the same no matter what kind of router we're hitting.

```python
# Iterate over all routers.
for hostname in hosts:
    # Perform a health check on the current router.
    try:
        status_code, response = get(hostname, "healthcheck")
        health = json.loads(response)
    except NotFoundError:
        print(f"Host {hostname} is not available")
        continue
    print(f"Host {hostname} is up")

    # Get the firmware version from the healthcheck data
    firmware_version = health["version"]
```

We have to do the login piece differently depending on the router's firmware version. This `if..else` block is going to get larger and larger as new firmware gets rolled out...

```python
    # We need to log in before we can do anything more than health check...
    if firmware_version == 1:
        status_code, response = post(
            hostname,
            "login",
            headers={"username": "admin", "password": "Password123"}
        )
    elif firmware_version == 2:
        status_code, response = post(
            hostname,
            "authenticate/admin",
            headers={"token": "token123456"}
        )
    if status_code == 200:
        print(f"  Logged in (v{firmware_version})")
    else:
        print(f"  Failed to log in")
        continue
    auth_token = response
```

When making the request for interface stats (or any other request we choose to do besides login and healthcheck), we have to know how to pass the authentication token in the request, again according to the firmware version.

```python
    # Now grab the interface stats.
    # We have to pass in the auth token because this is a protected operation.
    if firmware_version == 1:
        status_code, response = get(
            hostname, "interfaces", headers={"Token": auth_token}
        )
    elif firmware_version == 2:
        status_code, response = get(
            hostname,
            "interfaces",
            headers={"Authentication": f"TOKEN {auth_token}"}
        )
    if status_code != 200:
        print(f"  Failed to get interface stats: {response}")
        continue
    
    # Response is always JSON, so let's decode that.
    iface_stats = json.loads(response)
```

The way the data is returned is different, even though the result we want -- interface throughput stats -- is the same. We have to _normalize_ the response.

It's also quite possible that future firmware versions would differ in their requests and responses. For example, the throughput might be returned in `kbps` or `mbps`.

```python
    # Now let's figure out the throughput
    if firmware_version == 1:
        total_throughput = (
            iface_stats["eth0"]["up"] + iface_stats["eth0"]["down"]
        )
    elif firmware_version == 2:
        total_throughput = (
            int(iface_stats["eth0"]["up"].split()[0]) * 1000 +
            int(iface_stats["eth0"]["down"].split()[0]) * 1000
        )
```

tl;dr: our goal is simply to get throughput stats from a bunch of routers, but in our program, that purpose is obscured by all the other -- necessary -- logic.

We can use **Object Oriented Programming** principles to impose some structure and make things more manageable.

In the same order as above:

1. **Abstraction**: we can abstract away the logic used to obtain the information we want.
2. **Polymorphism**: _(meaning "many shapes")_ we can make all the routers _look_ the same to the core logic of our program by presenting operations with the same name that return their results in a normalized form (in this case, always in _numbers_ representing single bits per second).
3. **Inheritance**: `TutorRouterV1` and `TutorRouterV2` are both kinds of `TutorRouter`, so the commonalities (like the hostname property and healthcheck method) can be defined in the _"supertype"_ `TutorRouter`.
4. **Encapsulation**: rather than scattering logic for logging in, making calls and interpreting results; we can bundle them together with the data about each router.

To do this we will define _Classes_ to represent routers as _Objects_.

## Classes and objects

Speaking purely in the sense of human languages for a moment: notice in our scenarios that each router, e.g. `TutorRouter01`, is an _instance_ of a _class_ of devices we know as _Routers_. 

This thinking carries through to Object Oriented terminology.

A _class_ is like a template, from which we create _objects_ to represent individual _instances_ of the class.

> An **object** is an **instance** of a **class**.
>
> Confusingly, the term _object_ is often used interchangeably to refer to both _instances_ of a class and the _class_ itself.

To give a couple of examples:

* `bgillespie` is an instance of the class `Person`.
* `router1` is an instance of the class `Router`.

## Inheritance

When we speak of _inheritance_ in Object Oriented Programming (OOP) parlance, it's not the same as inheritance in the biological sense. In the OOP context, _inheritance_ is between _classes_ rather than _objects_ (instances). It's more accurate to think of _inheritance_ (in the OOP sense) as an _"is-a" relationship_.

e.g.:

* A `Human` or a `Whale` _is-a_ `Mammal`.
* A `Koala` or a `Wallaby` _is-a_ `Marsupial`.
* A `Router` or a `Switch` _is-a_ `NetworkDevice`.

On the other hand, `michelleyeoh` _isn't_ `janetyeoh`, even though they are both _instances_ of `Human`, so a `michelleyeoh` _object_ wouldn't inherit from a `janetyeoh` object any more than a `jamieleecurtis` object would, in the OOP sense.

> Note that unless it's strictly called for, it's often not desirable to define superclasses or class inheritance relations.

How does _inheritance_ apply to our scenario?

Since there's a lot in common between the different versions of router, let's define a **superclass** (also known as a _parent class_) to represent this notional common kind of router. Later, we'll define **subclasses** to represent each version of router.

We need to be careful to only add attributes that are common to **both** kinds of router. For now, it only has a `hostname` field.

```python
class TutorRouterClient:
    def __init__(self, hostname):
        self.hostname = hostname
```

> I've called this class `TutorRouter`_`Client`_ to guard against a dangerous illusion: that objects of this class represent things that respond reliably and instantaneously, as if they were wholly in-memory on your own machine (even though in the _simulation_, that's exactly what they are).
>
> Real routers are operated over the network and may not even respond at all. Calling the class _Client_ makes this clearer.

In a Python class definition, `__init__` is a special method used to initialize an object (instance) of a class. The `self` parameter refers to the object instance and is always the first parameter of _instance methods_. We use the `self` parameter to access the properties and methods of the instance.

In this case, the `self` parameter is used to define the `hostname` property on the new instance and initialize it with the value from the `hostname` parameter that will be passed in by callers.

As we'll see below, we won't be instantiating `TutorRouterClient` directly, nor calling its `__init__` method or any other of its methods directly. We'll come back to the superclass again a bit later.

## Polymorphism

In our script, we had logic that performed different operations depending on which kind of router was being addressed, but the _goals_ of those operations were the same, regardless of the version. It'd be nice to be able to make the same calls -- e.g. `get the throughput` -- against the objects and not have to worry about firmware versions.

What we're talking about is _Polymorphism_. Routers come in different _shapes_ but we can treat them more or less the same.

We can represent both kinds of router as different classes, each of which uses different _methods_ to attain the same goals.

```python
class TutorRouterV1Client(TutorRouterClient):
    """A client for a v1 TutorRouter."""
    
    def __init__(self, hostname):
        """Initialize a TutorRouterV1Client.
        
        Args:
            hostname: the router's hostname.
        """
        super().__init__(self, hostname)
        self._auth_token = None

    def login(self, username, password):
        """Log into the router.
        
        Args:
            username: the user we're logging in.
            password: the user's password.
            
        Raises:
            Exception: if there is an error whilst quthenticating.
        """
        status_code, response = post(
            self.hostname,
            "login",
            headers={"username": username, "password": password}
        )
        if status_code != 200:
            raise Exception(
                f"Error whilst authenticating {username} on {self.hostname}"
            )
        self._auth_token = response

        
class TutorRouterV2Client(TutorRouterClient):
    """A client for a v2 TutorRouter."""
    
    def __init__(self, hostname):
        """Initialize a TutorRouterV2Client.
        
        Args:
            hostname: the router's hostname.
        """
        super().__init__(self, hostname)
        self._token = None

        
    def login(self, username, password):
        """Log into the router.
        
        Args:
            username: the user we're logging in.
            password: the user's password.
            
        Raises:
            Exception: if there is an error whilst authenticating.
        """
        status_code, response = post(
            hostname,
            f"authenticate/{username}",
            headers={"token": password}
        )
        if status_code != 200:
            raise Exception(
                f"Error whilst authenticating {username} on {self.hostname}"
            )
        self._auth_token = response
```

In Python, a class inherits from another class by placing the superclass name in parentheses after the class name in the definition. Both new classes _inherit_ from `TutorRouterClient` and thus gain its properties and methods.

The methods' _docstrings_ note any `Arg`uments passed in, any exception(s) a method sometimes `Raises` and the data it `Returns`.

> This style of docstring comes from [the Google Python Style Guide](https://google.github.io/styleguide/pyguide.html#s3.8.1-comments-in-doc-strings).

The `__init__` method calls the superclass' `__init__` method to initialize the hostname, and defines a `_auth_token` property for the new instance.

Both classes have the `_auth_token` so why not define this in the superclass? It's quite possible that a future firmware would require an entirely different method of authenticating that's not token-based in the same way, so a token is not common to all current _and future_ subclasses. The same goes for the `POST` call for login: the method of logging in could change, so if that call was baked into the superclass, it could become redundant for some objects.

On the other hand, all routers now and in the future will have a `hostname`, so it makes sense to have that in the superclass.

Note that we don't need to check the firmware version before each call, because each class only needs to handle its own kind of calls and its own parsing logic.

## Encapsulation

Recall that we use the `healthcheck` to determine which version of router we're addressing. Where, then, is the best place for this logic?

Considering that these new classes will probably become part of a module, it's perfectly reasonable to provide a separate function to make the healthcheck call and return an object of the appropriate type: one of `TutorRouterV1Client` or `TutorRouterV2Client`.

Since the technique of determining the type of router is special to TutorRouters, it would also make sense to make this function a **static method** of `TutorRouterClient`.

> A **static method** is a method bound to a class, but does not operate directly on an _instance_ of a class.

By putting `instantiate` in `TutorRouterClient`, it **encapsulates** related code within the same logical unit that owns it.

In fact, everything we've done so far and will do from now involves Encapsulation, because all related methods and properties of an object are bound together in the same class.

```python
class TutorRouterClient:
    
    # ... the rest of the code here ...

    @staticmethod
    def instantiate(hostname):
        """Instantiate the correct subtype of TutorRouterClient.
        
        Makes a call to the `healthcheck` endpoint to determine
        the right subtype, according to the `version` field.
        
        Args:
            hostname: the router host name
        
        Returns:
            an initialized subtype of `TutorRouterClient`.
        
        Raises:
            Exception: if the router cannot be reached.
            Exception: if the healthcheck cannot be performed.
            Exception: if the version cannot be determined.
        """
        status_code, response = get(self.hostname, "healthcheck")
        if status_code != 200:
            raise Exception(
                f"Error whilst performing health check of {hostname}"
            )
        health = json.loads(response)
        
        try:
            version = health['version']
        except KeyError:
            raise Exception(
                f"Could not determine version of {hostname}"
            )
        
        if version == 1:
            return TutorRouterV1Client(hostname)
        elif version == 2:
            return TutorRouterV2Client(hostname)
        else:
            raise Exception(
                f"Version {version} of {hostname} not known"
            )
```

> Throughout I've been using `Exception` for all exceptions, just for simplicity. In reality it's better to create our own exception subtypes, but that's for another tutorial.

You would call this method like so:

```python
router = TutorRouterClient(hostname)
```

The type of `router` would be `TutorRouterV1Client` or `TutorRouterV2Client` depending on the healthcheck call. You could then call all the other methods on `router` as usual, provided an exception wasn't thrown.

## Abstraction

Users of our object are only interested in the throughput numbers. They don't care how we determine the firmware version, or what the routers return from the query. Users certainly aren't interested in finding out how to parse the response, nor will they want to deal with different return types.

As class authors, we need to _abstract away_ that logic...

```python
class TutorRouterV1Client:
    
    # ... the rest of the code so far ...
    
    def _get_interface_stats(self):
        """Return the interface stats reponse from the router.
        
        Raises:
            Exception: if we get a non-200 response.
            
        Returns:
            See the router API docs.
        """
        status_code, response = get(
            self.hostname,
            "interfaces",
            headers={"Token": self._auth_token}
        )
        if status_code != 200:
            raise Exception(
                "Failed to get interface stats"
            )
        return json.loads(response)
    
    def get_throughput(self) -> int:
        """Return the total throughput (up and down) to the router.
        
        Returns:
            the number of bits per second.
        """
        iface_stats = self._get_interface_stats()
        total_throughput = (
            iface_stats["eth0"]["up"] +
            iface_stats["eth0"]["down"]
        )
        return total_throughput


class TutorRouterV2Client:
    
    # ... the rest of the code so far ...
    
    def _get_interface_stats(self):
        """Return the interface stats reponse from the router.
        
        Raises:
            Exception: if we get a non-200 response.
            
        Returns:
            See the router API docs.
        """
        status_code, response = get(
            hostname,
            "interfaces",
            headers={"Authentication": f"TOKEN {auth_token}"}
        )
        if status_code != 200:
            raise Exception(
                "Failed to get interface stats"
            )
        return json.loads(response)
    
    def get_throughput(self) -> int:
        """Return the total throughput (up and down) to the router.
        
        Returns:
            the number of bits per second as an integer.
        """
        iface_stats = self._get_interface_stats()
        total_throughput = (
            int(iface_stats["eth0"]["up"].split()[0]) * 1000 +
            int(iface_stats["eth0"]["down"].split()[0]) * 1000
        )
        return total_throughput
```

Here we split the operation into two steps: getting the interface stats from the router, and calculating then returning the throughput. The latter method calls the former method.

### Public vs Private methods

The `_get_interface_stats` method's name starts with an underscore. This is a Python idiom indicating that the method is **private**, i.e. not for users of the class to call themselves. In truth there is nothing logically in Python to prevent the method from being called from anywhere, it's just idiom and linters that complain. We mark the method private because the output hasn't been sanitised for users, and is not guaranteed to return the same form of data, or even to be present in later versions of the code.

The `get_throughput` method is **public**, i.e. for user consumption, so it doesn't start with an underscore.

### Type annotations

The `-> int` at the end of the method declaration is a **type annotation** telling callers that the return type is `int`. This is also useful for IDEs and linters, which will warn you if the data returned is not the same type as the declaration, most likely indicating a bug.

It is also possible to mark the method's parameters with type annotations. For example, we could annotate the `login` method like so:

```python
    def login(self, username: str, token: str) -> "TutorRouterClient":
```

A few things to note:

* `self` is never annotated.
* The return type has to be in quotes because as of that point in the program, the class is not yet defined. (There are ways around this, but that's for another time).
* We're indicating that we're returning a `TutorRouterClient`, even though the actual return type is one of `TutorRouterV1Client` or `TutorRouterV2Client`, however that's valid because both a `TutorRouterV1Client` **is-a** `TutorRouterClient` and a `TutorRouterV2Client` **is-a** `TutorRouterClient`.

Onto [Putting It All Together](scenario_together.ipynb)...